# This file is used for descriptives of the outage data

In [7]:
import pandas as pd
from pathlib import Path
import zipfile
import re

In [8]:
# load data
wd = Path.cwd()
folder = 'stata'
file = 'Outages_{period}.dta'
zip = 'Outages_{period}.zip'
zip = zipfile.ZipFile(wd.parent/folder/zip)
data = pd.read_stata(zip.open(file)) # convert_categoricals = True
#print(data.columns)

# keep relevant columns
#data = data[['date', 'duration_minutes', 'discom', 'planned', 'noofcustomersaffected', 'unservedmuduetooutage', 'q']]
data['year'] = data.apply(lambda row: row.date.year, axis=1)

# keep year 2019
#data = data[data.apply(lambda row: row.date.year == 2019, axis=1)]

# keep only unplanned
data = data.loc[data.planned != 'planned',]

# at least 100 custumers affected
data = data.loc[data.noofcustomersaffected >= 100]

data.reset_index(drop = True)

# there is probably some rounding issue in unserveredmuduetooutage
# if it is 0 we replace it with 0.00001
data.loc[data.unservedmuduetooutage.isna() ,'unservedmuduetooutage'] = 0.00001


# keep one firm
#data = data.loc[data.discom == 'tata',]
data.describe()

,id,noofcustomersaffected,unservedmuduetooutage,hour_of_day,duration_seconds,duration_minutes,duration_hours,industrial,school,hospital,dairy,residential,minute_of_day,q,quarter,hourly,year
count,90073.000000,91230.000000,91230.000000,91230.000000,91230.000000,91230.000000,91230.000000,91230.000000,91230.000000,91230.000000,91230.000000,91230.000000,91230.000000,91230.000000,91230.000000,91230.000000,91230.000000
mean,468.618521,1473.176324,0.000639,12.517867,3769.354072,62.822568,1.047043,0.048690,0.057459,0.016464,0.021638,0.032489,28.942377,2.478779,24711.152344,8401.883789,2018.356725
std,522.128451,1717.586345,0.001219,5.606701,3977.549315,66.292489,1.104875,0.215303,0.232840,0.127211,0.145528,0.177272,17.357660,1.119374,14567.838867,4894.696289,0.604226
min,1.000000,100.000000,-0.013772,0.000000,60.000000,1.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,2017.000000
25%,96.000000,305.000000,0.000100,9.000000,1897.000000,31.616667,0.526944,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,1.000000,12115.250000,4262.000000,2018.000000
50%,275.000000,845.000000,0.000289,13.000000,2940.000000,49.000000,0.816667,0.000000,0.000000,0.000000,0.000000,0.000000,29.000000,2.000000,23535.000000,7819.000000,2018.000000
75%,639.000000,2019.000000,0.000756,17.000000,4380.000000,73.000000,1.216667,0.000000,0.000000,0.000000,0.000000,0.000000,44.000000,3.000000,37715.000000,12836.000000,2019.000000
max,2986.000000,15574.000000,0.070739,23.000000,245340.000000,4089.000000,68.150000,1.000000,1.000000,1.000000,1.000000,1.000000,59.000000,4.000000,51343.000000,17572.000000,2019.000000


Reason of outages: list of most occuring words

In [9]:
# tidy up 
data['remark_short'] = data.apply(lambda row: row.remarks.replace('\n', ' '), axis = 1)
data.loc[:, 'remark_short'] = data.apply(lambda row: row.remark_short.replace('faults', 'fault'), axis = 1)
data.loc[:, 'remark_short'] = data.apply(lambda row: row.remark_short.replace('trafic', 'traffic'), axis = 1)
data.loc[:, 'remark_short'] = data.apply(lambda row: row.remark_short.replace('tripping', 'trip'), axis = 1)
data.loc[:, 'remark_short'] = data.apply(lambda row: row.remark_short.replace('tripped', 'trip'), axis = 1)
data.loc[:, 'remark_short'] = data.apply(lambda row: row.remark_short.replace('/', ' '), axis = 1)
data.loc[:, 'remark_short'] = data.apply(lambda row: row.remark_short.replace(',', ' '), axis = 1)
data.loc[:, 'remark_short'] = data.apply(lambda row: row.remark_short.replace('(', ''), axis = 1)
data.loc[:, 'remark_short'] = data.apply(lambda row: row.remark_short.replace(')', ''), axis = 1)
data.loc[:, 'remark_short'] = data.apply(lambda row: row.remark_short.strip(), axis = 1)
data.loc[:, 'remark_short'] = data.apply(lambda row: re.sub(' +', ' ',row.remark_short), axis = 1)

# make all remarks in list
remark_list = data.remark_short.tolist()
for i in range(len(remark_list)):
    remark_list[i] = remark_list[i].split()

# list of all words
remark_list_new=[]
for x in remark_list: 
    for i in x:
        i = i.lower() # all lower case
        remark_list_new.append(i)

remark_list = remark_list_new


words_to_remove = ['outage', 'due', 'to', 'and', 'or', 'with', 'for', '.', 'at', 'interruption','schedule', 'on', 'of','system']
# list of words that should not be considered
remark_list = [ w for w in remark_list if w not in words_to_remove]

# list of unique words
remark_unique = list(set(remark_list)) 

# now could the occurences of unique words and save in dictionary
remark_dict = {}
for i in remark_unique:
    remark_dict[i] = remark_list.count(i)

# get most
remark_most = sorted(remark_dict, key = remark_dict.get, reverse=True)[0:20]

# here you can simply add words you want in the table
words_to_count = ['trip', 'storm', 'birdage', 'tree', 'water','load', 'monkey'] 
for w in words_to_count:
    remark_most.append(w)

# extract number of occurences from dict and save in list
remark_most_values = []
for i in remark_most:
    remark_most_values.append(remark_dict[i])

# new dictionary with only the most words
remark_dict = {}
for i in range(len(remark_most)):
    remark_dict[remark_most[i]] = remark_most_values[i]

# put dictionary in df
df = pd.DataFrame.from_dict(remark_dict,orient='index', columns = ['occurences']).reset_index()
df = df.rename(columns={'index':'remark'})
total_remarks = data.remarks.count()

#df['share'] = df.occurences / total_remarks
#df.sort_values(by = ['occurences'])
#print(df)
# export df to latex
#df.to_latex('remarks.tex', index=False)